<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/agents_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Preparando el cuaderno**
---
bla, bla, bla

In [ ]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.
%pip install langgraph -qU  # Instalar la librería de grafos de LangChain.

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


# # Importamos las clases necesarias para trabajar con cadenas
# from langchain.chains import LLMChain

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint

# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

In [ ]:
from typing import Dict, List, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph

# 1. Definir el estado del grafo
class TranslationState(TypedDict):
    original_text: str
    source_language: str
    target_language: str
    translator_output: str
    reviewer_suggestions: List[str]
    editor_output: str
    translation_iterations: int

# 2. Definir las plantillas
translator_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "Eres un lingüista experto, especializado en traducción del lenguaje "
        "{source_language} al {target_language}. Utiliza técnicas avanzadas para "
        "proporcionar traducciones fluidas y precisas."
    )),
    ("human", (
        "Traduce el siguiente texto del {source_language} al {target_language}:\n"
        "{original_text}"
    ))
])

reviewer_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "Eres un lingüista experto, especializado en traducción del lenguaje "
        "{source_language} al {target_language}. Se te proporcionará un texto original "
        "y una traducción. Tu objetivo es mejorar la traducción y dar sugerencias. "
        "Si la traducción está perfectamente bien y no se requieren cambios, "
        "devuelve exactamente la palabra 'NO_SUGERENCIAS'."
    )),
    ("human", (
        "Texto original ({source_language}):\n{original_text}\n\n"
        "Traducción del traductor:\n{translator_output}\n\n"
        "Proporciona sugerencias y críticas constructivas:"
    ))
])

editor_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "Eres un lingüista experto, especializado en traducción del {source_language} "
        "al {target_language}. Tu tarea es leer y editar la traducción final, "
        "aplicando las sugerencias relevantes."
    )),
    ("human", (
        "Traducción del traductor:\n{translator_output}\n\n"
        "Sugerencias del revisor:\n{reviewer_suggestions}\n\n"
        "Realiza la edición final de la traducción:"
    ))
])

# 3. Configurar el modelo
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)

# 4. Crear runnables
translator_chain = translator_prompt | llm
reviewer_chain = reviewer_prompt | llm
editor_chain = editor_prompt | llm

# 5. Definir funciones de nodo
def translate(state: TranslationState) -> dict:
    print("\n--- TRADUCIENDO ---")
    result = translator_chain.invoke(state)
    print(f"Traducción inicial: {result.content}")
    return {"translator_output": result.content}

def review(state: TranslationState) -> dict:
    print("\n--- REVISANDO ---")
    result = reviewer_chain.invoke(state)
    content = result.content.strip()
    if content == "NO_SUGERENCIAS":
        suggestions = []
    else:
        suggestions = [line.strip() for line in content.split("\n") if line.strip()]
    print(f"Sugerencias del revisor: {suggestions}")
    return {"reviewer_suggestions": suggestions}

def edit(state: TranslationState) -> dict:
    print("\n--- EDITANDO ---")
    result = editor_chain.invoke(state)
    print(f"Edición final: {result.content}")
    return {"editor_output": result.content}

def decide_next_step(state: TranslationState) -> dict:
    max_iterations = 3
    suggestions = state.get("reviewer_suggestions", [])
    current_iter = state["translation_iterations"]  # Leer el valor actual

    # SOLO si hay sugerencias y no se ha llegado al tope:
    if suggestions and current_iter < max_iterations:
        current_iter += 1
        print(f"\nEl revisor ha hecho sugerencias. Iteración n.º {current_iter}. Volvemos a traducir.")
        return {
            "translation_iterations": current_iter,
            "decision": "translate"
        }
    else:
        print("\nEl revisor no ha hecho sugerencias o se alcanzó el máximo de iteraciones. Pasando a 'edit'.")
        return {
            "translation_iterations": current_iter,
            "decision": "edit"
        }

# 6. Construir el grafo
builder = StateGraph(TranslationState)

builder.add_node("translate", translate)
builder.add_node("review", review)
builder.add_node("edit", edit)
builder.add_node("check_review", decide_next_step)

builder.set_entry_point("translate")

builder.add_edge("translate", "review")
builder.add_edge("review", "check_review")

builder.add_conditional_edges(
    "check_review",
    lambda out: out["decision"],  # cómo leer la decisión
    {
        "translate": "translate",
        "edit": "edit"
    }
)

builder.add_edge("edit", END)
graph = builder.compile()

# 8. Ejecutar
inputs = {
    "original_text": "Los Angeles firefighters were making modest progress in taming the region’s two largest fires on Saturday as they raced to suppress them ahead of high winds that were expected to intensify later in the day. After a night of expanded evacuation orders and spreading flames that continued to plunge the area into what Lindsey Horvath, a Los Angeles County supervisor, called 'unimaginable terror and heartbreak,' crews had contained 11 percent of the 22,660-acre Palisades fire and 15 percent of the 14,000-acre Eaton fire, near Altadena and Pasadena, according to Cal Fire.",
    "source_language": "inglés",
    "target_language": "español",
    "translation_iterations": 0
}
result = graph.invoke(inputs)

# 9. Resultado
print("\n--- RESULTADO FINAL ---")
print(f"Texto original: {result['original_text']}")
print(f"Idioma de origen: {result['source_language']}")
print(f"Idioma de destino: {result['target_language']}")
print(f"Traducción final: {result.get('editor_output')}")
